In [5]:
import json

from collections import Counter

In [2]:
with open('../public/static/bike_routes.geojson') as fh:
    data = json.loads(fh.read())

In [86]:
with open('../public/static/bike_routes.geojson', 'w+') as fh:
    fh.write(json.dumps(data))

In [3]:
categories = Counter()
lengths = Counter()

for feature in data['features']:
    categories[feature['properties']['bike_network_category']] += 1
    if feature['geometry']:
        lengths[len(feature.get('geometry', {}).get('coordinates'))] += 1

In [4]:
categories

Counter({'Proposed Bike+ - Non-Arterial': 3076,
         'Proposed Bike+ - Arterial': 1503,
         'Proposed Bike+ - Upgrade, Arterial': 1464,
         'Existing Bike+ - Non-Arterial': 804,
         'Existing Bike+ - Arterial': 525,
         'Existing Multi-Use Trail': 337,
         'Non-Bike+ Planned': 250,
         'Non-Bike+': 197,
         'Proposed Multi-Use Trail': 175,
         '': 136})

In [21]:
!pip install pyhigh

     |████████████████████████████████| 19.8MB 16.2MB/s eta 0:00:01
  Running setup.py install for pyhigh ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
data['features'][0]

{'type': 'Feature',
 'id': 1,
 'geometry': {'type': 'LineString',
  'coordinates': [[-122.308954277912, 47.604327437898],
   [-122.307649047488, 47.6043216339769]]},
 'properties': {'OBJECTID': 1,
  'id': 23677,
  'artclass_desc': 'Not Designated',
  'stname_ord': 'E ALDER ST',
  'bike_network_category': 'Proposed Bike+ - Non-Arterial'}}

In [8]:
from pyhigh import get_elevation

/Users/abrahamepton/venvs/maps/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [29]:
get_elevation(lat=47.604327437898, lon=-122.308954277912) * 3.28084

314.96064

In [28]:
get_elevation(lat=47.6043216339769, lon=-122.307649047488) * 3.28084

321.52232

In [27]:
get_elevation(lat=47.5541164, lon=-122.2896943) * 3.28084

144.35696

In [44]:
pairs = {}
min_elev = 0
max_elev = 0

for feature in data['features']:
    if not feature['geometry']:
        continue
    idx = 0
    while idx < len(feature['geometry']['coordinates']) - 1:
        origin = feature['geometry']['coordinates'][idx]
        dest = feature['geometry']['coordinates'][idx + 1]
        origin_elev = get_elevation(lat=origin[1], lon=origin[0])
        dest_elev = get_elevation(lat=dest[1], lon=dest[0])
        pair = f"{origin[0]},{origin[1]} - {dest[0]},{dest[1]}"
        pairs[pair] = abs(dest_elev - origin_elev)
        max_elev = max(pairs[pair], max_elev)
        min_elev = min(pairs[pair], min_elev)
        idx += 1

In [46]:
print(max_elev, min_elev)

73.0 0.0


In [29]:
for p in pairs:
    print(p, pairs[p])
    break

-122.308954277912,47.604327437898 - -122.307649047488,47.6043216339769 2.0


In [9]:
def get_color(val, max_elev=73.0, min_elev=0.0):
    color_scale = [
        '#309143',
        '#51b364',
        '#8ace7e',
        '#ffda66',
        '#f0bd27',
        '#e39802',
        '#ff684c',
        '#e03531',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c',
        '#b60a1c'
    ]
    max_min_range = max_elev - min_elev
    intervals = max_min_range / (len(color_scale) - 1)
    return color_scale[int(val/intervals)]

In [84]:
output = {
    'type': 'FeatureCollection',
    'crs': { 'type': 'name', 'properties': {'name': 'EPSG:4326'}},
    'features': []
}
for p in pairs:
    color = get_color(pairs[p], max_elev, min_elev)
    origin, dest = p.split(' - ')
    origin_lon, origin_lat = origin.split(',')
    dest_lon, dest_lat = dest.split(',')
    output['features'].append({
        "type": "Feature",
        "id": 1,
        "geometry": {
            "type": "LineString",
            "coordinates":[[float(origin_lon), float(origin_lat)], [float(dest_lon), float(dest_lat)]]
        },
        "properties": {'line-color': color}
    })

In [85]:
with open('../public/static/bike_routes_heights.geojson', 'w+') as fh:
    fh.write(json.dumps(output))

In [6]:
with open('../public/static/seattle_streets_raw.geojson') as fh:
    raw_streets_data = json.loads(fh.read())

In [7]:
raw_streets_data['features'][0]

{'type': 'Feature',
 'id': 2367059,
 'geometry': {'type': 'LineString',
  'coordinates': [[-122.322859636095, 47.5298142840705],
   [-122.322873008817, 47.5290153917875]]},
 'properties': {'OBJECTID': 2367059,
  'F_INTR_ID': 17740,
  'T_INTR_ID': 17791,
  'SND_ID': 33478,
  'SND_FEACODE': 5,
  'CITYCODE': 1,
  'STNAME_ID': 566,
  'ST_CODE': 0,
  'ARTERIAL_CODE': 1,
  'SEGMENT_TYPE': 1,
  'AGENCY_CODE': 1,
  'ACCESS_CODE': 1,
  'DIVIDED_CODE': 1,
  'STRUCTURE_TYPE': 1,
  'LEGALLOC_CODE': 1,
  'VEHICLE_USE_CODE': 1,
  'GIS_SEG_LENGTH': 291.426597,
  'L_ADRS_FROM': 8200,
  'L_ADRS_TO': 8298,
  'R_ADRS_FROM': 8201,
  'R_ADRS_TO': 8299,
  'ORD_PRE_DIR': None,
  'ORD_STREET_NAME': '8TH',
  'ORD_STREET_TYPE': 'AVE',
  'ORD_SUF_DIR': 'S',
  'ORD_STNAME_CONCAT': '8TH AVE S',
  'L_CITY': 'SEATTLE',
  'L_STATE': 'WA',
  'L_ZIP': '98108',
  'R_CITY': 'SEATTLE',
  'R_STATE': 'WA',
  'R_ZIP': '98108',
  'SNDSEG_UPDATE': 'Sun, 22 Apr 2007 00:00:00 GMT',
  'COMPKEY': 8460,
  'COMPTYPE': 68,
  'UNITID'

In [17]:
street_pairs = {}
min_elev = 0
max_elev = 0

for feature in raw_streets_data['features']:
    if not feature['geometry']:
        continue
    idx = 0
    while idx < len(feature['geometry']['coordinates']) - 1:
        origin = feature['geometry']['coordinates'][idx]
        dest = feature['geometry']['coordinates'][idx + 1]
        origin_elev = get_elevation(lat=origin[1], lon=origin[0])
        dest_elev = get_elevation(lat=dest[1], lon=dest[0])
        pair = f"{origin[0]},{origin[1]} - {dest[0]},{dest[1]}"
        street_pairs[pair] = abs(dest_elev - origin_elev)
        max_elev = max(street_pairs[pair], max_elev)
        min_elev = min(street_pairs[pair], min_elev)
        idx += 1
        if len(street_pairs) % 1000 == 0:
            print(f'Found {len(street_pairs):,} pairs')

Found 1,000 pairs
Found 2,000 pairs
Found 3,000 pairs
Found 4,000 pairs
Found 5,000 pairs
Found 6,000 pairs
Found 7,000 pairs
Found 8,000 pairs
Found 9,000 pairs
Found 10,000 pairs
Found 11,000 pairs
Found 12,000 pairs
Found 13,000 pairs
Found 14,000 pairs
Found 15,000 pairs
Found 16,000 pairs
Found 17,000 pairs
Found 18,000 pairs
Found 19,000 pairs
Found 20,000 pairs
Found 21,000 pairs
Found 22,000 pairs
Found 23,000 pairs
Found 24,000 pairs
Found 25,000 pairs
Found 26,000 pairs
Found 27,000 pairs
Found 28,000 pairs
Found 29,000 pairs
Found 30,000 pairs
Found 31,000 pairs
Found 32,000 pairs
Found 33,000 pairs
Found 34,000 pairs
Found 35,000 pairs
Found 36,000 pairs
Found 37,000 pairs
Found 38,000 pairs
Found 39,000 pairs
Found 40,000 pairs
Found 41,000 pairs
Found 42,000 pairs
Found 43,000 pairs
Found 44,000 pairs
Found 45,000 pairs
Found 46,000 pairs
Found 47,000 pairs
Found 48,000 pairs
Found 49,000 pairs
Found 50,000 pairs
Found 51,000 pairs
Found 52,000 pairs
Found 53,000 pairs
Fo

In [18]:
output = {
    'type': 'FeatureCollection',
    'crs': { 'type': 'name', 'properties': {'name': 'EPSG:4326'}},
    'features': []
}
for p in street_pairs:
    color = get_color(street_pairs[p], max_elev, min_elev)
    origin, dest = p.split(' - ')
    origin_lon, origin_lat = origin.split(',')
    dest_lon, dest_lat = dest.split(',')
    output['features'].append({
        "type": "Feature",
        "id": 1,
        "geometry": {
            "type": "LineString",
            "coordinates":[[float(origin_lon), float(origin_lat)], [float(dest_lon), float(dest_lat)]]
        },
        "properties": {'line-color': color}
    })

In [19]:
with open('../public/static/streets_heights.geojson', 'w+') as fh:
    fh.write(json.dumps(output))